In [ ]:
import numpy as np
import xarray as xr
import pyvista as pv
import matplotlib.pyplot as plt

import os

PATH = '/Volumes/GoogleDrive/My Drive/Segyio and PyVista'

pv.rcParams['use_panel'] = False

In [ ]:
def read_tiff_surface(filename):
    """Read a geotiff and make a surface mesh with PyVista

    Helpful: http://xarray.pydata.org/en/stable/auto_gallery/plot_rasterio.html
    """
    # Read in the data
    data = xr.open_rasterio(filename)
    values = np.asarray(data)
    nans = values == data.nodatavals
    if np.any(nans):
        # values = np.ma.masked_where(nans, values)
        values[nans] = np.nan
    # Make a mesh
    xx, yy = np.meshgrid(data['x'], data['y'])
    zz = values.reshape(xx.shape)
    # zz = np.zeros_like(xx)
    mesh = pv.StructuredGrid(xx, yy, zz)
    mesh[os.path.basename(filename)] = values.ravel(order='F')
    return mesh


In [ ]:
filename = os.path.join(PATH, 'teapot_dem.tif')
teapot_dem = read_tiff_surface(filename)
teapot_dem

In [ ]:
teapot_dem.plot()

In [ ]:
def associate_tiff_texture(mesh, filename):
    """Read a tiff and associated it as a texture on the given mesh
    This does everything in place on the mesh"""
    data = xr.open_rasterio(filename)
    values = np.asarray(data)
    # Swap array order
    values = values.swapaxes(0, -1).swapaxes(0,1)
    if values.shape[-1] == 1:
        # Convert Grayscale to RGB
        values = np.stack((values.reshape(values.shape[0:2]),)*3, axis=-1)
    print('Texture image shape: ', values.shape)
    # Get spatial reference
    xx, yy = np.meshgrid(data['x'], data['y'])
    extent = np.min(xx), np.max(xx), np.min(yy), np.max(yy)
    origin = (extent[0], extent[2], 0.0)  # BOTTOM LEFT CORNER
    point_u = (extent[1], extent[2], 0.0) # BOTTOM RIGHT CORNER
    point_v = (extent[0], extent[3], 0.0) # TOP LEFT CORNER
    # Map the texuture
    name = os.path.basename(filename)
    teapot_dem.texture_map_to_plane(origin, point_u, point_v, inplace=True, name=name)
    teapot_dem.textures[name] = pv.numpy_to_texture(values)
    return None

In [ ]:
filename = os.path.join(PATH, 'Topo_NPR3.tif')
associate_tiff_texture(teapot_dem, filename)

In [ ]:
filename = os.path.join(PATH, 'teapot_aerial_NAD27.tif')
associate_tiff_texture(teapot_dem, filename)

teapot_dem

In [ ]:
# Warp by scalar filter majorly exaggerates the topo
teapot_dem.warp_by_scalar(factor=5.).plot(texture='teapot_aerial_NAD27.tif', 
                                          notebook=False, )